In [1]:
import tensorflow as tf
from tensorflow import keras

fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.4 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

In [5]:
train_images = train_images.astype('float32')/255.0
test_images = test_images.astype('float32')/255.0

In [7]:
#definiowanie modelu z hiperparametrami
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  hp_learnig_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learnig_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model


In [8]:
#utworzenie wystąpienia hypertunera
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='mojkatalog',
                     project_name='intro_to_kt')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [10]:
#wyszukiwanie hiperparametrów
tuner.search(train_images, train_labels, epochs=50, validation_split=0.2, callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Liczba neuronów: {best_hps.get('units')}
Szybkość uczenia: {best_hps.get('learning_rate')}
""")


Trial 30 Complete [00h 01m 55s]
val_accuracy: 0.8799166679382324

Best val_accuracy So Far: 0.8947499990463257
Total elapsed time: 00h 19m 40s

Liczba neuronów: 416
Szybkość uczenia: 0.001



In [11]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_images, train_labels, epochs=50, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7833 - loss: 0.6134 - val_accuracy: 0.8432 - val_loss: 0.4367
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8629 - loss: 0.3839 - val_accuracy: 0.8615 - val_loss: 0.3751
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.8796 - loss: 0.3284 - val_accuracy: 0.8773 - val_loss: 0.3362
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.8829 - loss: 0.3155 - val_accuracy: 0.8830 - val_loss: 0.3265
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8918 - loss: 0.2893 - val_accuracy: 0.8844 - val_loss: 0.3198
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.8971 - loss: 0.2725 - val_accuracy: 0.8828 - val_loss: 0.3341
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.9019 - loss: 0.2651 - val_accuracy: 0.8837 - val_loss: 0.3313
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.9079 - loss: 0.

In [12]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Liczba epok:', best_epoch)

Liczba epok: 26


In [13]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(train_images, train_labels, epochs=best_epoch, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/26
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.7800 - loss: 0.6261 - val_accuracy: 0.8457 - val_loss: 0.4238
Epoch 2/26
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.8626 - loss: 0.3777 - val_accuracy: 0.8648 - val_loss: 0.3664
Epoch 3/26
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8801 - loss: 0.3266 - val_accuracy: 0.8775 - val_loss: 0.3498
Epoch 4/26
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8869 - loss: 0.3086 - val_accuracy: 0.8838 - val_loss: 0.3180
Epoch 5/26
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.8952 - loss: 0.2839 - val_accuracy: 0.8758 - val_loss: 0.3540
Epoch 6/26
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.9010 - loss: 0.2708 - val_accuracy: 0.8840 - val_loss: 0.3311
Epoch 7/26
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.9060 - loss: 0.2520 - val_accuracy: 0.8730 - val_loss: 0.3565
Epoch 8/26
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9063 - loss: 0

In [14]:
#ocena hipermodelu
eval_result = hypermodel.evaluate(test_images, test_labels)
print("[test loss, test accuracy]:", eval_result)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8861 - loss: 0.4067
[test loss, test accuracy]: [0.4058504104614258, 0.8920999765396118]
